In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from stepmix import StepMix

In [2]:
reduced = pd.read_csv('dimension_reduction.csv')

In [3]:
df = reduced.loc[:, 'adulterant':'weak opioid']
df.head()

,adulterant,amphetamine & derivatives,anabolic,anesthetic,anti-anxiety,anti-tussive,benzodiazepine,bulking agent,cannabinoid,cocaine impurity,...,semi-synthetic,steroid,substituted cathinone,supplement,sweetener,synthetic cannabinoid,topical/dermal,up-and-down,upper,weak opioid
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X= df.to_numpy()

model = StepMix(n_components=4, measurement='binary', verbose=1)
model.fit(df)

# Predict latent class memberships
labels = model.predict(X)

# Get class probabilities - this is the probabilty that each substance is in each class 234 substances x 5 classes
# probs = model.predict_proba(X)

# # Print result
# print("Labels:", labels)
# print("Probabilities:\n", probs)


# # Model parameters
# print("Parameters:\n", model.get_parameters())

Fitting StepMix...


Initializations (n_init) : 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.13it/s, max_LL=-2.13e+3, max_avg_LL=-9.1]

MODEL REPORT
    Measurement model parameters
          model_name                       binary                        
          class_no                              0       1       2       3
          param variable                                                 
          pis   adulterant                 0.0000  0.0955  0.2131  0.0671
                amphetamine & derivatives  0.0000  0.0000  0.0000  0.3225
                anabolic                   0.2198  0.0000  0.0000  0.0000
                anesthetic                 0.0000  0.1032  0.0161  0.0000
                anti-anxiety               0.0000  0.0000  0.2684  0.0000
                anti-tussive               0.0000  0.0561  0.0000  0.0000
                benzodiazepine             0.0000  0.0000  0.2349  0.0000
                bulking agent              0.0000  0.1589  0.0000  0.0000
                cannabinoid                0.0000  0.0000  0.3523  0.0000
                cocaine impurity           0.0000  0.0841  0.0000 

In [ ]:
# Get the feature names from your original DataFrame
feature_names = df.columns.tolist()

# Create a mapping DataFrame
feature_map_df = pd.DataFrame({
    'features': [f'feature_{i}' for i in range(len(feature_names))],
    'feature_name': feature_names
})

# Display the mapping
print(feature_map_df)

In [ ]:
#put column and the pis values into a df
#print(emission_probs)

feature_order = [
    "feature_0", "feature_1", "feature_10", "feature_11", "feature_12", "feature_13",
    "feature_14", "feature_15", "feature_16", "feature_17", "feature_18", "feature_19",
    "feature_2", "feature_20", "feature_21", "feature_22", "feature_23", "feature_24",
    "feature_25", "feature_26", "feature_27", "feature_28", "feature_29", "feature_3",
    "feature_30", "feature_31", "feature_32", "feature_33", "feature_34", "feature_35",
    "feature_36", "feature_37", "feature_38", "feature_39", "feature_4", "feature_40",
    "feature_41", "feature_42", "feature_43", "feature_44", "feature_45", "feature_46",
    "feature_47", "feature_48", "feature_5", "feature_6", "feature_7", "feature_8",
    "feature_9"
]

In [ ]:
df = pd.DataFrame(emission_probs).T  # or: pd.DataFrame(np.array(data).T)
df['features'] = feature_order
df

In [ ]:
merged_df = pd.merge(df, feature_map_df, on='features')
merged_df[['feature_name', 0, 1, 2, 3]]

In [ ]:
# Step 1: Get conditional probabilities
emission_probs = model.get_parameters()["measurement"]["pis"]  # shape: (n_classes, n_features)
feature_names = df.columns.tolist()  # works if X is a DataFrame

n_classes, n_features = emission_probs.shape

# Build a DataFrame for each class
class_tables = []
for class_idx in range(n_classes):
    probs = emission_probs[class_idx, :]  # shape: (n_features,)
    df = pd.DataFrame({
        "Feature": feature_names,
        f"Prob_Class_{class_idx}": probs
    }).sort_values(by=f"Prob_Class_{class_idx}", ascending=False)
    class_tables.append(df)

class_tables